In [1]:
import pandas as pd
from unidecode import unidecode
from kiblib.utils.db import DbConn

In [2]:
year = 2024
db_conn = DbConn().create_engine()

# Préparation des données

In [3]:
acq_colsname = {
        'itemnumber': 'Code barre',
        'Collection': 'Nom de la collection',
        'CollectiviteAuteur': 'Champ Collectivité auteur',
        'DateEdition': 'Date d’édition',
        'DatesAuteur': 'Dates de l’auteur',
        'Dewey': 'Indice Dewey',
        'NomAuteur': 'Nom de l’auteur',
        'NumeroVolume': 'Numéro de volume',
        'PrenomAuteur': 'Prénom de l’auteur',
        'SousTitre': 'Sous-titre',
        'TitreSerie': 'Titre de série',
        'TitreVolume': 'Titre du volume',
        'dateaccessioned' : 'Date d’acquisition'
    }

acq_cols = ['Titre', 'Numéro de volume', 'Titre du volume', 'ISBN', 'Code barre', 'Sous-titre', 'Date d’édition', 'Nom de la collection', 'Nom de l’auteur', 'Prénom de l’auteur', 'Dates de l’auteur', 'Champ Collectivité auteur', 'Titre de série', 'Indice Dewey', 'Date d’acquisition']

In [8]:
prets_colsname = {
    'itemnumber': 'Code barre',
    'Collection': 'Nom de la collection',
    'CollectiviteAuteur': 'Champ Collectivité auteur',
    'DateEdition': 'Date d’édition',
    'DatesAuteur': 'Dates de l’auteur',
    'Dewey': 'Indice Dewey',
    'NomAuteur': 'Nom de l’auteur',
    'NumeroVolume': 'Numéro de volume',
    'PrenomAuteur': 'Prénom de l’auteur',
    'SousTitre': 'Sous-titre',
    'TitreSerie': 'Titre de série',
    'TitreVolume': 'Titre du volume',
    'issuedate': 'Date d’emprunt'
}

prets_cols = ['Titre', 'Numéro de volume', 'Titre du volume', 'ISBN', 'Code barre', 'Sous-titre', 'Date d’édition', 'Nom de la collection', 'Nom de l’auteur', 'Prénom de l’auteur', 'Dates de l’auteur', 'Champ Collectivité auteur', 'Titre de série', 'Indice Dewey', 'Date d’emprunt']

In [18]:
# attention, au préalable : déposer dans data le dernier dumps Koha (se trouve dans koha_prod/data/dumps_koha/notices)
# et lancer sur celui-ci le script baro_marc2csv.py
# Voir https://gist.github.com/ragbx/0cb52b49ace2a3fbd33ab791bb41612a

# dataframe comprenant l'ensemble des notices bibliographiques en fin d'année
biblio = pd.read_csv("../data/2025-02-23-notices_total.csv")
biblio['biblionumber'] = biblio['biblionumber'].astype(str)

/tmp/ipykernel_27327/2387876382.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  biblio = pd.read_csv("../data/2025-02-23-notices_total.csv")


In [5]:
# dataframe contenant toutes les acquisitions de l'année
request = f"""SELECT
  i.biblionumber,
  i.itemnumber,
  i.dateaccessioned,
  bi.itemtype,
  i.ccode,
  i.itemcallnumber
FROM koha{year}.items i
JOIN koha{year}.biblioitems bi ON i.biblionumber = bi.biblionumber
WHERE
  YEAR(i.dateaccessioned) = {year}
"""
acq = pd.read_sql(request, db_conn)
acq = acq[~acq['biblionumber'].isna()]
acq['biblionumber'] = acq['biblionumber'].astype(int).astype(str)
acq = acq[~acq['itemnumber'].isna()]
acq['itemnumber'] = acq['itemnumber'].astype(int).astype(str)
len(acq)

15879

In [6]:
# dataframe contenant tous les prêts de l'année
request = f"""SELECT
  issuedate,
  biblionumber,
  itemnumber,
  itemtype,
  ccode,
  itemcallnumber
FROM statdb.stat_issues
WHERE
  YEAR(issuedate) = {year}
"""
prets = pd.read_sql(request, db_conn)
prets = prets[~prets['biblionumber'].isna()]
prets['biblionumber'] = prets['biblionumber'].astype(int).astype(str)
prets = prets[~prets['itemnumber'].isna()]
prets['itemnumber'] = prets['itemnumber'].astype(int).astype(str)
len(prets)

364268

# Bandes dessinées

In [13]:
acq_bd = acq[acq['itemtype'].isin(['LI', 'LV', 'LS'])]
acq_bd1 = acq_bd[acq_bd['ccode'].isin(['AAPBDZZ', 'AAPBDCM', 'AAPBDSR', 'AAPBDER', 'AAPBDMG', 'JBDZZZZ',
                                       'JBDCLZZ', 'JBDCMZZ', 'JBDMGZZ', 'JBDSRZZ', 'JBDPBZZ'])]
acq_bd2 = acq_bd[acq_bd['itemcallnumber'].str[0:2] == 'BD']
acq_bd3 = acq_bd[acq_bd['itemcallnumber'].str[0:5] == 'FR/BD']

acq_bd = pd.concat([acq_bd1, acq_bd2, acq_bd3])
acq_bd = acq_bd.drop_duplicates()
print(f"acq BD avant merge : {len(acq_bd)}")

acq_bd = acq_bd.merge(biblio, on="biblionumber", how="inner")
acq_bd = acq_bd.rename(columns=acq_colsname)
acq_bd = acq_bd[acq_cols]
acq_bd['Code lieu'] = 'LI2550'
file_out = f"../data/RBX_Barometre{year}_Acquisitions_BD.csv.gz"
acq_bd.to_csv(file_out, header=True, index=False)
print(f"acq BD : {len(acq_bd)}")


prets_bd = prets[prets['itemtype'].isin(['LI', 'LV', 'LS'])]
prets_bd1 = prets_bd[prets_bd['ccode'].isin(['AAPBDZZ', 'AAPBDCM', 'AAPBDSR', 'AAPBDER', 'AAPBDMG', 'JBDZZZZ',
                                             'JBDCLZZ', 'JBDCMZZ', 'JBDMGZZ', 'JBDSRZZ', 'JBDPBZZ'])]
prets_bd2 = prets_bd[prets_bd['itemcallnumber'].str[0:2] == 'BD']
prets_bd3 = prets_bd[prets_bd['itemcallnumber'].str[0:5] == 'FR/BD']

prets_bd = pd.concat([prets_bd1, prets_bd2, prets_bd3])
prets_bd = prets_bd.drop_duplicates()
print(f"prets BD avant merge : {len(prets_bd)}")

prets_bd = prets_bd.merge(biblio, on="biblionumber", how="inner")
prets_bd = prets_bd.rename(columns=prets_colsname)
prets_bd = prets_bd[prets_cols]
prets_bd['Code lieu'] = 'LI2550'
file_out = f"../data/RBX_Barometre{year}_Prets_BD.csv.gz"
prets_bd.to_csv(file_out, header=True, index=False)
print(f"prets BD : {len(prets_bd)}")

acq BD avant merge : 2050
acq BD : 2049
prets BD avant merge : 103467
prets BD : 102912


# Documentaires

In [14]:
acq_doc = acq[acq['itemtype'].isin(['LI', 'LV', 'LS'])]
acq_doc1 = acq_doc[acq_doc['ccode'].isin(['AAPATAP', 'AAPBDDC', 'AAPCGDA', 'AAPCGDD', 'AAPCIDC', 'AAPCIFI',
                                          'AAPPRLD', 'AAPATLC', 'AAPPLCB', 'AAPPLFF', 'AAPPLGO', 'AAPPLHU',
                                          'AAPPLMS', 'AAPPLPA', 'ACFPDZZ', 'ACFPMZZ', 'AAPPLPS', 'AAPPLSL',
                                          'AAPCITV', 'ACFBPZZ', 'ACFLAFF', 'ACFHSZZ', 'ACFIFZZ', 'ACFLALE',
                                          'ACFPHZZ', 'ACFPAZZ', 'ACFREZZ', 'ACFRUZZ', 'ACFSCZZ', 'ACFSSZZ',
                                          'ACFUEZZ', 'ALTLSDC', 'AMVASCA', 'AMVASDC', 'AMV11ZZ', 'AMV80ZZ',
                                          'AMV13ZZ', 'AMV30ZZ', 'AMV39ZZ', 'AMV50ZZ', 'AMV00ZZ', 'AMV25ZZ',
                                          'AMV15ZZ', 'AMV16ZZ', 'AMV20ZZ', 'AMV14ZZ', 'AMV40ZZ', 'AMVTHCA',
                                          'AMVTHDC', 'AMVTHTX', 'AMV29ZZ', 'AMV90ZZ', 'AZZMAZZ', 'JDTFDZZ'])]
acq_doc2 = acq_doc[acq_doc['itemcallnumber'].str[0:7] == 'ROUBAIX']
acq_doc3 = acq_doc[~acq_doc['itemcallnumber'].isna()]
acq_doc3 = acq_doc3[acq_doc3['itemcallnumber'].str.contains('^FR/[0-9]', regex=True)]

acq_doc = pd.concat([acq_doc1, acq_doc2, acq_doc3])
acq_doc = acq_doc.drop_duplicates()
print(f"acq Doc avant merge : {len(acq_doc)}")

acq_doc = acq_doc.merge(biblio, on="biblionumber", how="inner")
acq_doc = acq_doc.rename(columns=acq_colsname)
acq_doc = acq_doc[acq_cols]
acq_doc['Code lieu'] = 'LI2550'
file_out = f"../data/RBX_Barometre{year}_Acquisitions_Documentaires.csv.gz"
acq_doc.to_csv(file_out, header=True, index=False)
print(f"acq Doc : {len(acq_doc)}")

prets_doc = prets[prets['itemtype'].isin(['LI', 'LV', 'LS'])]
prets_doc1 = prets_doc[prets_doc['ccode'].isin(['AAPATAP', 'AAPBDDC', 'AAPCGDA', 'AAPCGDD', 'AAPCIDC', 'AAPCIFI',
                                          'AAPPRLD', 'AAPATLC', 'AAPPLCB', 'AAPPLFF', 'AAPPLGO', 'AAPPLHU',
                                          'AAPPLMS', 'AAPPLPA', 'ACFPDZZ', 'ACFPMZZ', 'AAPPLPS', 'AAPPLSL',
                                          'AAPCITV', 'ACFBPZZ', 'ACFLAFF', 'ACFHSZZ', 'ACFIFZZ', 'ACFLALE',
                                          'ACFPHZZ', 'ACFPAZZ', 'ACFREZZ', 'ACFRUZZ', 'ACFSCZZ', 'ACFSSZZ',
                                          'ACFUEZZ', 'ALTLSDC', 'AMVASCA', 'AMVASDC', 'AMV11ZZ', 'AMV80ZZ',
                                          'AMV13ZZ', 'AMV30ZZ', 'AMV39ZZ', 'AMV50ZZ', 'AMV00ZZ', 'AMV25ZZ',
                                          'AMV15ZZ', 'AMV16ZZ', 'AMV20ZZ', 'AMV14ZZ', 'AMV40ZZ', 'AMVTHCA',
                                          'AMVTHDC', 'AMVTHTX', 'AMV29ZZ', 'AMV90ZZ', 'AZZMAZZ', 'JDTFDZZ'])]
prets_doc2 = prets_doc[prets_doc['itemcallnumber'].str[0:7] == 'ROUBAIX']
prets_doc3 = prets_doc[~prets_doc['itemcallnumber'].isna()]
prets_doc3 = prets_doc3[prets_doc3['itemcallnumber'].str.contains('^FR/[0-9]', regex=True)]

prets_doc = pd.concat([prets_doc1, prets_doc2, prets_doc3])
prets_doc = prets_doc.drop_duplicates()
print(f"prets doc avant merge : {len(prets_doc)}")

prets_doc = prets_doc.merge(biblio, on="biblionumber", how="inner")
prets_doc = prets_doc.rename(columns=prets_colsname)
prets_doc = prets_doc[prets_cols]
prets_doc['Code lieu'] = 'LI2550'
file_out = f"../data/RBX_Barometre{year}_Prets_Documentaires.csv.gz"
prets_doc.to_csv(file_out, header=True, index=False)
print(f"prets doc : {len(prets_doc)}")

acq Doc avant merge : 1516
acq Doc : 1515
prets doc avant merge : 29644
prets doc : 29112


# Fiction

In [15]:
acq_fiction = acq[acq['itemtype'].isin(['LI', 'LV', 'LS'])]
acq_fiction1 = acq_fiction[acq_fiction['ccode'].isin(['AAPPRLP', 'ALTTCCN', 'ALTDCZZ', 'ALTIGFA', 'ALTLSZZ',
                                                      'ALTLSCN', 'ALTLSRO', 'ALTLSPL', 'ALTTCNV', 'ALTPOZZ',
                                                      'ALTROZZ', 'ALTROAD', 'ALTROAM', 'ALTROPL', 'ALTIGSF',
                                                      'ALTLVZZ', 'ALTTLZZ'])]
acq_fiction3 = acq_fiction[~acq_fiction['itemcallnumber'].isna()]
acq_fiction3 = acq_fiction3[acq_fiction3['itemcallnumber'].str.contains('^FR/R', regex=True)]

acq_fiction = pd.concat([acq_fiction1, acq_fiction3])
acq_fiction = acq_fiction.drop_duplicates()
print(f"acq Fiction avant merge : {len(acq_fiction)}")

acq_fiction = acq_fiction.merge(biblio, on="biblionumber", how="inner")
acq_fiction = acq_fiction.rename(columns=acq_colsname)
acq_fiction = acq_fiction[acq_cols]
acq_fiction['Code lieu'] = 'LI2550'
file_out = f"../data/RBX_Barometre{year}_Acquisitions_Fiction.csv.gz"
acq_fiction.to_csv(file_out, header=True, index=False)
print(f"acq Fiction : {len(acq_fiction)}")

prets_fiction = prets[prets['itemtype'].isin(['LI', 'LV', 'LS'])]
prets_fiction1 = prets_fiction[prets_fiction['ccode'].isin(['AAPPRLP', 'ALTTCCN', 'ALTDCZZ', 'ALTIGFA', 'ALTLSZZ',
                                                            'ALTLSCN', 'ALTLSRO', 'ALTLSPL', 'ALTTCNV', 'ALTPOZZ',
                                                            'ALTROZZ', 'ALTROAD', 'ALTROAM', 'ALTROPL', 'ALTIGSF',
                                                            'ALTLVZZ', 'ALTTLZZ'])]
prets_fiction3 = prets_fiction[~prets_fiction['itemcallnumber'].isna()]
prets_fiction3 = prets_fiction3[prets_fiction3['itemcallnumber'].str.contains('^FR/R', regex=True)]

prets_fiction = pd.concat([prets_fiction1, prets_fiction3])
prets_fiction = prets_fiction.drop_duplicates()
print(f"prets Fiction avant merge : {len(prets_fiction)}")

prets_fiction = prets_fiction.merge(biblio, on="biblionumber", how="inner")
prets_fiction = prets_fiction.rename(columns=prets_colsname)
prets_fiction = prets_fiction[prets_cols]
prets_fiction['Code lieu'] = 'LI2550'
file_out = f"../data/RBX_Barometre{year}_Prets_Fiction.csv.gz"
prets_fiction.to_csv(file_out, header=True, index=False)
print(f"prets Fiction : {len(prets_fiction)}")

acq Fiction avant merge : 1641
acq Fiction : 1641
prets Fiction avant merge : 46163
prets Fiction : 45810


# Jeunesse

In [16]:
acq_jeunesse = acq[acq['itemtype'].isin(['LI', 'LV', 'LS'])]
acq_jeunesse = acq_jeunesse[acq_jeunesse['ccode'].str[0] == 'J']
acq_jeunesse = acq_jeunesse[~acq_jeunesse['ccode'].str.contains('^JBD', regex=True)]

acq_jeunesse = acq_jeunesse.drop_duplicates()
print(f"acq Jeunesse avant merge : {len(acq_jeunesse)}")

acq_jeunesse = acq_jeunesse.merge(biblio, on="biblionumber", how="inner")
acq_jeunesse = acq_jeunesse.rename(columns=acq_colsname)
acq_jeunesse = acq_jeunesse[acq_cols]
acq_jeunesse['Code lieu'] = 'LI2550'
file_out = f"../data/RBX_Barometre{year}_Acquisitions_Jeunesse.csv.gz"
acq_jeunesse.to_csv(file_out, header=True, index=False)
print(f"acq Jeunesse : {len(acq_jeunesse)}")

prets_jeunesse = prets[prets['itemtype'].isin(['LI', 'LV', 'LS'])]
prets_jeunesse = prets_jeunesse[prets_jeunesse['ccode'].str[0] == 'J']
prets_jeunesse = prets_jeunesse[~prets_jeunesse['ccode'].str.contains('^JBD', regex=True)]

prets_jeunesse = prets_jeunesse.drop_duplicates()
print(f"prets Jeunesse avant merge : {len(prets_jeunesse)}")

prets_jeunesse = prets_jeunesse.merge(biblio, on="biblionumber", how="inner")
prets_jeunesse = prets_jeunesse.rename(columns=prets_colsname)
prets_jeunesse = prets_jeunesse[prets_cols]
prets_jeunesse['Code lieu'] = 'LI2550'
file_out = f"../data/RBX_Barometre{year}_Prets_Jeunesse.csv.gz"
prets_jeunesse.to_csv(file_out, header=True, index=False)
print(f"prets Jeunesse : {len(prets_jeunesse)}")

acq Jeunesse avant merge : 3934
acq Jeunesse : 3934
prets Jeunesse avant merge : 118389
prets Jeunesse : 117855
